In [1]:
import pandas as pd, numpy as np

from matplotlib import pyplot as plt
import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%a\n%m/%d')

import seaborn as sns

In [3]:
path = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
covid = pd.read_csv(path)
covid['date'] = pd.to_datetime(covid.date)

path = 'http://www.wtad.com/assets/news_files/PEP_2018_PEPANNCHG.ST05_with_ann.xlsx'
pop = pd.read_excel(path)

pop = pop.rename(columns={'GC.target-geo-id2':'fips',
                'respop72018':'population'})

pop = pop[['fips','population']]

pop = pop.loc[1:]

pop['population'] = pd.to_numeric(pop['population'])
pop['fips'] = pd.to_numeric(pop.fips)

In [4]:
path = "https://raw.githubusercontent.com/dri3s/covidviz/master/metrolist.csv"
msa = pd.read_csv(path, nrows=1160, header=None, names=['msa_fips','msa_name','fips','county_name'])
msa.fips = pd.to_numeric(msa['fips'])
msa = msa.drop(columns=['msa_fips','county_name'])

In [21]:
msa['city'] = msa.msa_name.str.split(pat=',|-', expand=True)[[0]]

In [30]:
msasplit = msa.msa_name.str.split(pat=',|-', expand=True)

In [31]:
msasplit

0                                    1         2     3  \
0        Abilene   TX (Metropolitan Statistical Area)      None  None   
1        Abilene   TX (Metropolitan Statistical Area)      None  None   
2        Abilene   TX (Metropolitan Statistical Area)      None  None   
3          Akron   OH (Metropolitan Statistical Area)      None  None   
4          Akron   OH (Metropolitan Statistical Area)      None  None   
...          ...                                  ...       ...   ...   
1155  Youngstown                               Warren  Boardman    OH   
1156  Youngstown                               Warren  Boardman    OH   
1157   Yuba City   CA (Metropolitan Statistical Area)      None  None   
1158   Yuba City   CA (Metropolitan Statistical Area)      None  None   
1159        Yuma   AZ (Metropolitan Statistical Area)      None  None   

                                       4     5     6  
0                                   None  None  None  
1                                   None  None  None  
2                                   None  None  None  
3                                   None  None  None  
4                                   None  None  None  
...                                  ...   ...   ...  
1155  PA (Metropolitan Statistical Area)  None  None  
1156  PA (Metropolitan Statistical Area)  None  None  
1157                                None  None  None  
1158                                None  None  None  
1159                                None  None  None  

[1160 rows x 7 columns]

In [34]:
for idx, r in msasplit.iterrows():
    pass

In [47]:
r[1].strip()[0:2]

'AZ'

In [ ]:
covidmsa = covid.merge(pop).merge(msa)


# In[]

covidmsa = covidmsa.drop(columns='fips').groupby(['date','msa_name']).sum().reset_index()

# In[]



covidmsa['casepercap'] = covidmsa.cases/covidmsa['population']
covidmsa = covidmsa.sort_values(by=['msa_name','date']).reset_index(drop=True)
covidmsa = covidmsa.loc[covidmsa.cases >= 10]


# In[]


covidmsa ['num_days'] = (covidmsa.date - covidmsa.groupby(['msa_name'])['date'].transform('min')).dt.days


# In[]



worst = list(covidmsa.groupby('msa_name').casepercap.max().reset_index().sort_values('casepercap').tail(10).msa_name.values)



# In[]

dets = list(covidmsa.loc[covidmsa.msa_name.str.contains('Dallas')].msa_name.unique())
dets.append(covidmsa.loc[covidmsa.msa_name.str.contains('Houston')].msa_name.unique()[0])
dets.append(covidmsa.loc[covidmsa.msa_name.str.contains('College Station')].msa_name.unique()[0])


# In[]


colors = iter(sns.color_palette('husl', len(worst) + len(dets)))

plt.rcParams['figure.figsize'] = 15, 5
fig, ax = plt.subplots()

for msaname, df in covidmsa.groupby('msa_name'):          
    df = df.sort_values(by='num_days')
    
    if msaname in worst or msaname in dets:
        label = msaname
        color = next(colors)
        lw, alpha = (3, 1)    
    else:        
        label = '_nolabel_'
        color='gray'
        lw, alpha = (0.4, 0.4)
        
    if msaname in dets:
        ls = ':'
        lw = 5
    else:
        ls = '-'
    
    #ax.plot(covidmsa.loc[covidmsa.fips == fips]['date'],
    ax.plot(df['num_days'],
            df['casepercap'] * 1e3, 
            label=label,
            color=color,
            lw=lw,
            alpha=alpha,
            ls = ls
            )

#ax.set_xlim([pd.to_datetime('15feb2020'),pd.to_datetime(date.today())])
#ax.xaxis.set_major_formatter(myFmt)
#ax.xaxis.set_major_locator(mdates.WeekdayLocator(mdates.MO))

ax.set_xlim([0, covidmsa.num_days.max()])
ax.set_yscale('log')

ax.set_ylabel('covidmsa Cases per 1,000 people')
ax.set_xlabel('Num Days Since 10th Case')
ax.legend()
ax.grid()




# In[]


np.log(8)/np.log(2)


# In[]